In [2]:
import pandas as pd
import numpy as np
import folium
import datetime
import datetime as dt
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
import requests

I've extracted a .csv of all the earthquakes logged by the SSN in Mexico from 2018 to Nov 11, 2020 with magnitude greater than or equal to four.

http://www2.ssn.unam.mx:8080/catalogo/

Next, we're going to cross reference this data with the OpenEEW device data from AWS to get device readings from the given earthquake times.

https://s3.amazonaws.com/grillo-openeew/index.html

We've got about 9000 earthquakes. Also, the typical earthquake lasts between 10-30 seconds. To account for travel time, I'm going record 5 minutes after the reported time of a quake. Each second, a given device records 32 accelerometer readings on the x,y, and z axis. To keep the data from being 50 gigabytes, I'm going to average each seconds readings into a single value for the axis.

So, our problem is, given recordings 5 minutes of accelerometer data from multiple locations, how accurately can we triangulate an epicenter? Conventional wisdom states that accelerometers are poor for measuring distance, but what about over local area, given fixed placements, and with the extra data given from a 5 minute waveform? 

In [3]:
quakes_df=pd.read_csv("..\..\data\\all_quakes.csv")
quakes_df['utc_datetime']= quakes_df['date_utc'] + " " + quakes_df['time_utc']
quakes_df.drop(columns=['date_utc', 'time_utc'], inplace=True)
quakes_df['utc_datetime'] = pd.to_datetime(quakes_df['utc_datetime'])
quakes_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime
0,4.0,15.0648,-94.4530,16.3,"135 km al SUROESTE de TONALA, CHIS",2018-01-01 07:15:42
1,4.2,16.2913,-95.3580,64.5,"13 km al SUROESTE de TEHUANTEPEC, OAX",2018-01-01 08:32:09
2,4.3,15.2057,-94.5715,18.4,"127 km al SURESTE de SALINA CRUZ, OAX",2018-01-01 10:05:06
3,4.0,15.9422,-92.8240,199.8,"50 km al NORESTE de PIJIJIAPAN, CHIS",2018-01-01 14:33:18
4,4.0,15.5092,-94.8927,30,"81 km al SURESTE de SALINA CRUZ, OAX",2018-01-01 14:59:54
...,...,...,...,...,...,...
8989,5.0,14.1100,-92.1000,26,"63 km al SUR de CD HIDALGO, CHIS",2020-11-12 03:21:46
8990,4.3,15.2200,-94.5900,17,"124 km al SURESTE de SALINA CRUZ, OAX",2020-11-12 04:48:21
8991,4.1,14.4700,-92.3500,80,"32 km al SUROESTE de CD HIDALGO, CHIS",2020-11-12 05:55:58
8992,4.0,17.4700,-92.7600,10,"23 km al SURESTE de TEAPA, TAB",2020-11-12 08:24:41


In [7]:
quakes_df.shape

(8994, 6)

In [4]:
#this devices log was obtained from an earlier query through the busted OpenEEW API.

devices=pd.read_csv("..\..\data\devices.csv")

In [9]:
devices

,country_code,device_id,latitude,longitude,effective_from,effective_to,is_current_row,vertical_axis,horizontal_axes
0,mx,1,15.67,-96.50,1483228800,253402300799,True,x,"['y', 'z']"
1,mx,2,15.86,-97.07,1521655200,253402300799,True,x,"['y', 'z']"
2,mx,4,16.35,-98.05,1483228800,253402300799,True,x,"['y', 'z']"
3,mx,5,16.44,-95.02,1521676800,253402300799,True,x,"['y', 'z']"
4,mx,6,16.68,-98.40,1483228800,253402300799,True,x,"['y', 'z']"
5,mx,7,16.32,-95.24,1521817200,253402300799,True,x,"['y', 'z']"
6,mx,8,16.61,-98.98,1483228800,253402300799,True,x,"['y', 'z']"
7,mx,9,16.72,-99.12,1483228800,253402300799,True,x,"['y', 'z']"
8,mx,10,16.79,-99.39,1521559800,253402300799,True,x,"['y', 'z']"
9,mx,11,16.84,-99.90,1483228800,253402300799,True,x,"['y', 'z']"


Structure: We're gonna append a column for each of the devices to the quake table. For each cell, we'll insert a 3x300 numpy array of the avg x, y, and z values per second for a five minute interval after the earthquake. There's gonna be lots of nulls -- for now, we'll just give those the 0 vector and sort them out later.

In [10]:
# initialize waveform columns
for device in devices['device_id']:
    col_name="device_"+str(device)
    quakes_df[col_name]=np.zeros(quakes_df.shape[0])

In [11]:
quakes_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,device_1,device_2,device_4,device_5,...,device_20,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999
0,4.0,15.0648,-94.4530,16.3,"135 km al SUROESTE de TONALA, CHIS",2018-01-01 07:15:42,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.2,16.2913,-95.3580,64.5,"13 km al SUROESTE de TEHUANTEPEC, OAX",2018-01-01 08:32:09,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.3,15.2057,-94.5715,18.4,"127 km al SURESTE de SALINA CRUZ, OAX",2018-01-01 10:05:06,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,15.9422,-92.8240,199.8,"50 km al NORESTE de PIJIJIAPAN, CHIS",2018-01-01 14:33:18,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,15.5092,-94.8927,30,"81 km al SURESTE de SALINA CRUZ, OAX",2018-01-01 14:59:54,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8989,5.0,14.1100,-92.1000,26,"63 km al SUR de CD HIDALGO, CHIS",2020-11-12 03:21:46,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8990,4.3,15.2200,-94.5900,17,"124 km al SURESTE de SALINA CRUZ, OAX",2020-11-12 04:48:21,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8991,4.1,14.4700,-92.3500,80,"32 km al SUROESTE de CD HIDALGO, CHIS",2020-11-12 05:55:58,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8992,4.0,17.4700,-92.7600,10,"23 km al SURESTE de TEAPA, TAB",2020-11-12 08:24:41,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I eventually realized that my script that gathered this data would take 70 hours to run for the above dataframe. So we're restricting this now to the top 1000 earthquakes by magnitude.

In [6]:
top_1000=quakes_df.sort_values('magnitude',  ascending=False).head(1000)

top_1000.reset_index(drop=True, inplace=True)

In [14]:
top_1000

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,device_1,device_2,device_4,device_5,...,device_20,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999
0,7.4,15.8033,-96.1337,22.8,"4 km al NORTE de CRUCECITA, OAX",2020-06-23 15:29:03,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.2,16.2180,-98.0135,16,"14 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-16 23:39:39,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.5,14.5818,-92.4807,76.2,"37 km al SUROESTE de CD HIDALGO, CHIS",2019-02-01 16:14:12,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.4,13.7792,-93.3382,9.4,"162 km al SUROESTE de CD HIDALGO, CHIS",2019-11-20 04:27:06,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.3,26.6672,-111.0790,16.1,"77 km al NORESTE de LORETO, BCS",2018-01-19 16:17:45,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4.3,15.5190,-94.9390,21.5,"78 km al SURESTE de SALINA CRUZ, OAX",2019-03-28 09:08:36,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,4.3,14.5795,-93.7258,20.1,"131 km al SUROESTE de MAPASTEPEC, CHIS",2019-02-25 19:38:56,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,4.3,14.4045,-93.9323,12.5,"160 km al SUROESTE de MAPASTEPEC, CHIS",2019-02-25 19:36:46,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,4.3,19.0497,-107.4690,10,"291 km al SUROESTE de PUERTO VALLARTA, JAL",2019-03-30 06:54:28,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
m = folium.Map(location=[15.8033, -96.1337],
              zoom_start=7)
for i, quake in top_1000.iterrows():
    folium.Circle(radius=1_000, 
                  location=[quake['latitude'], quake['longitude']], 
                  tooltip="Magnitude: "+ str(quake['magnitude']) + "\n" + "UTC DateTime: " + str(quake['utc_datetime']), 
                  color='crimson', fill='crimson').add_to(m)

for i, d in devices.iterrows():
        folium.Marker([d['latitude'], d['longitude']],
                      tooltip="Device ID :" + str(d['device_id']), sticky=True).add_to(m)

Plotting the top 1000 earthquakes by latitude and longitude gives us the following map.

In [8]:
m

Next, we write a few functions that we use to build a data scraping script for the AWS database. Record are uploaded 5 minutes at a time, at the 0, 5, ...55 marks of the hour. Since the 5 minute interval around an earthquake will probably span two of these files, we need to grab the files corresponding to the 10 minute intervals that surround it, merge those, and then find the relevant 5 minutes.

In [18]:
#given a UTC time, returns a tuple of the nearest 5 minute markers.
def eq_range(timestamp):
    lb = timestamp - dt.timedelta(minutes=timestamp.minute % 5, seconds=timestamp.second)
    ub = lb + dt.timedelta(minutes=5)
    return (lb, ub)

In [19]:
#returns [avg_x, avg_y, avg_z] array of the first 300 vals after a given time.
#we'll call this a waveform for lack of better terminology
def make_waveform(df, timestamp, device):
    #lower bound is 1 sec before recorded earthquake start
    lb= timestamp-dt.timedelta(seconds=1)
    df=df[df['device_id']==device]
    #return first 300 rows
    df_5min=df[df['device_t'] > lb].iloc[0:300, :]
    avg_x_vec=df_5min['avg_x'].values
    avg_y_vec=df_5min['avg_y'].values
    avg_z_vec=df_5min['avg_z'].values
    #assemble array
    return np.array([avg_x_vec, avg_y_vec, avg_z_vec])

In [20]:
#gets 10 minutes of data for a given earthquake time

def get_eq_data(eq_time):
    timestamps=eq_range(eq_time)
    df_list=[]
    for device in devices['device_id']:
        device_id = str(device).zfill(3)
        for timestamp in timestamps:
            year = timestamp.strftime('%Y')
            month= timestamp.strftime('%m')
            day = timestamp.strftime('%d')
            hour = timestamp.strftime('%H')
            minute = timestamp.strftime('%M')
            key=f'records/country_code=mx/device_id={device_id}/year={year}/month={month}/day={day}/hour={hour}/{minute}.jsonl'
            address='s3://grillo-openeew/' + key
            try:
                five_min_df=pd.read_json(address, lines=True)
                df_list.append(five_min_df)
            except:
                continue
    df=pd.concat(df_list)
    df.drop(columns=['country_code'], inplace=True)
    df['device_t']=pd.to_datetime(df['device_t'], unit='s')
    df['cloud_t']=pd.to_datetime(df['cloud_t'], unit='s')
    df['avg_x']= [np.mean(x_vec) for x_vec in df['x']]
    df['avg_y']= [np.mean(y_vec) for y_vec in df['y']]
    df['avg_z']= [np.mean(z_vec) for z_vec in df['z']]
    return df

In [21]:
#given quake df, gets waveforms for all devices.
#if data not found for a device, MARK IT ZERO

def get_all_data(df):
    #initialize lists to store waveforms for each device.
    num_devices=devices.shape[0]
    pickle_counter=0
    waveform_list=[[] for device in range(num_devices)]
    for eq_time in tqdm(df['utc_datetime']):
        eq_df=get_eq_data(eq_time)
        device_counter=0
        for device in devices['device_id']:
            if device in eq_df['device_id'].unique():
                waveform = make_waveform(eq_df, eq_time, device)
                waveform_list[device_counter].append(waveform)
                print("waveform added")
            else:
                waveform_list[device_counter].append(np.zeros((3,300)))
                print("zeros added")
            device_counter+=1
        pickle_counter+=1
        if pickle_counter % 50 == 0:
            pickle.dump(waveform_list, open( "backup.p", "wb" ))
    device_counter=0
    for device in devices['device_id']:
        col_name="device_"+str(device)
        df[col_name]=waveform_list[device_counter]
        device_counter +=1
    return df

In [24]:
final_df=get_all_data(top_1000)

  0%|                                                                               | 1/1000 [00:31<8:45:19, 31.55s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
waveform added
zeros added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
zeros added
zeros added
zeros added


  0%|▏                                                                              | 2/1000 [01:04<8:50:06, 31.87s/it]

waveform added
zeros added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added


  0%|▏                                                                              | 3/1000 [01:37<8:56:26, 32.28s/it]

...

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added


 84%|██████████████████████████████████████████████████████████████▊            | 837/1000 [8:02:36<2:45:54, 61.07s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added


 84%|██████████████████████████████████████████████████████████████▊            | 837/1000 [8:13:40<1:36:08, 35.39s/it]


ValueError: No objects to concatenate

In [ ]:
last_200=get_all_data(top_1000.iloc[800:, :])

  0%|▍                                                                               | 1/200 [00:35<1:57:07, 35.31s/it]

waveform added
zeros added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
zeros added
waveform added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added


  1%|▊                                                                               | 2/200 [01:05<1:51:00, 33.64s/it]

zeros added
zeros added
zeros added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
zeros added
zeros added
waveform added
waveform added
zeros added
waveform added
zeros added
zeros added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
zeros added
zeros added
zeros added


  2%|█▏                                                                              | 3/200 [01:35<1:47:25, 32.72s/it]

...

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added


 28%|██████████████████████                                                         | 56/200 [32:15<1:27:19, 36.39s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
zeros added
zeros added
waveform added


 28%|██████████████████████▌                                                        | 57/200 [32:51<1:26:38, 36.35s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added


In [23]:
last_150=get_all_data(top_1000.iloc[850:, :])

  1%|▌                                                                               | 1/150 [00:34<1:25:22, 34.38s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added


  1%|█                                                                               | 2/150 [01:09<1:25:12, 34.55s/it]

waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
waveform added
waveform added
waveform added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added


  2%|█▌                                                                              | 3/150 [01:48<1:28:15, 36.02s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▍| 149/150 [1:21:08<00:30, 30.52s/it]

waveform added
waveform added
waveform added
zeros added
waveform added
waveform added
zeros added
waveform added
zeros added
waveform added
zeros added
zeros added
waveform added
waveform added
zeros added
zeros added
waveform added
zeros added
waveform added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added
zeros added


100%|██████████████████████████████████████████████████████████████████████████████| 150/150 [1:21:40<00:00, 32.67s/it]

waveform added
waveform added
zeros added
zeros added
waveform added
waveform added
waveform added
waveform added
zeros added
waveform added
zeros added
zeros added
waveform added
zeros added
zeros added
waveform added
waveform added
zeros added
waveform added
waveform added
zeros added
zeros added
zeros added
waveform added
zeros added
zeros added
zeros added
waveform added



<ipython-input-21-5ce8c4ebece5>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name]=waveform_list[device_counter]


There were some issues with this process as you can guess from the above function calls. My internet went down a few times, among other things. I backed up the data after every 50 earthquakes and ended up merging it all together after a few calls for different ranges.

In [82]:
pickle.dump(final_df, open( "final_df.p", "wb" ))

In [83]:
final_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,device_1,device_2,device_4,device_5,...,device_20,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999
0,7.4,15.8033,-96.1337,22.8,"4 km al NORTE de CRUCECITA, OAX",2020-06-23 15:29:03,"[[0.00625, -0.0215625, -0.0009374999999999997,...","[[-0.0040625, -0.00625, 0.0006250000000000006,...","[[-0.027500000000000004, -0.005, 0.00624999999...","[[-0.015, -0.0053124999999999995, 0.0100000000...",...,"[[0.0025000000000000005, -0.009374999999999998...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.010624999999999999, -0.007500000000000001...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,7.2,16.2180,-98.0135,16,"14 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-16 23:39:39,"[[0.1018125, 0.106625, 0.10521875, 0.09146875,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.26025, 0.2543125, 0.25834375, 0.25059375, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.023968750000000004, 0.02040625, 0.0188125,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,6.5,14.5818,-92.4807,76.2,"37 km al SUROESTE de CD HIDALGO, CHIS",2019-02-01 16:14:12,"[[-0.035625000000000004, -0.0396875, -0.013437...","[[0.004687499999999999, 0.014375000000000002, ...","[[0.14437500000000003, 0.14250000000000002, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[-0.031875, -0.0225, -0.025, -0.01875, -0.019...","[[0.265625, 0.275625, 0.255625, 0.25625, 0.255...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.083125, 0.08875, 0.0871875, 0.0728125, 0.0..."
3,6.4,13.7792,-93.3382,9.4,"162 km al SUROESTE de CD HIDALGO, CHIS",2019-11-20 04:27:06,"[[0.0003125000000000016, -0.000312499999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.0028125000000000008, -0.0184375, 0.001250...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.015, -0.018437500000000002, 0.00062499999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.0146875, -0.0159375, -0.00874999999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0018750000000000004, 0.010625, -0.00375000...","[[-0.0006249999999999997, -0.02687500000000000...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,6.3,26.6672,-111.0790,16.1,"77 km al NORESTE de LORETO, BCS",2018-01-19 16:17:45,"[[0.042374999999999996, 0.052000000000000005, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.1003125, 0.10928125, 0.08753125, 0.1129375...",...,"[[-0.049, -0.02653125, -0.0336875, -0.02640625...","[[-0.026875000000000003, -0.045687500000000006...","[[0.05475, 0.0746875, 0.07171875000000001, 0.0...","[[0.18840625, 0.19443749999999999, 0.18971875,...","[[